# Gensim 

gensim是一种通过检查词汇模式（或更高级别的结构，如语句或文档）来发现文档语义结构（Semantic Structure）的工具。 gensim通过语料库 --- 一组文本文档，并在语料库中生成文本的向量表示（Vector Representation of the Text）来实现这一点。 然后，文本的向量表示可用于训练模型 --- 它是用于创建不同的文本数据（蕴含语义）表示的算法。

相关概念：

1. 语料（Corpus）：一组原始文本的集合，用于无监督地训练文本主题的隐层结构。在Gensim中，Corpus通常是一个可迭代的对象（比如列表）。每    一次迭代返回一个可用于表达文本对象的稀疏向量。

2. 向量（Vector）：由一组文本特征构成的列表。是一段文本在Gensim中的内部表达。在向量空间模型中，每个文档被表示成了一组特征，比如，一个    单一的特征可能被视为一个问答对。

3. 稀疏向量（SparseVector）：通常，大部分问题的答案都是0，为了节约空间，我们会从文档表示中省略他们，向量中的每一个元素是一个(key,      value)的元组，比如（1,3），（2,4），（5,0），其中（5,0）是一个稀疏向量，在表示是会被忽略。

4. 模型（Model）：是一个抽象的术语。定义了两个向量空间的变换（即从文本的一种向量表达变换为另一种向量表达）。


把几个概念组织起来表述：gensim可以通过读取一段语料，输出一个向量，表示文档中的一个词。为了节约空间，通常稀疏的词向量会被忽略，剩下的词向量则可以用来训练各种模型，即从原有的文本表达转向另一种文本表达。

In [2]:
import json
import jieba
import gensim
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim import corpora, models, similarities
from collections import defaultdict

/Users/zhaohaonan/anaconda3/lib/python3.7/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


## 定义 function，利用 Jieba 给句子分词

In [3]:
# 分词函数，返回分词列表(jieba)
# string -> string
def trans(x):
    x = list(jieba.cut(x))
    return " ".join(x)


### Jieba Example

In [4]:
s = '王八蛋老板黄鹤和他的小姨子跑了'
print(trans(s))

# "他的" 是分开的

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/x1/gwtt25l1289_g68tqjk3626c0000gn/T/jieba.cache
Loading model cost 0.832 seconds.
Prefix dict has been built succesfully.


王八蛋 老板 黄鹤 和 他 的 小姨子 跑 了


In [5]:
# "他的"合并
jieba.suggest_freq('他的', True)
print(trans(s))

王八蛋 老板 黄鹤 和 他的 小姨子 跑 了


# 1: 语料（Corpus）


一个语料库是数字文档的集合（A Collection of Digital Documents）。 
这个集合是gensim的输入，它将从中推断文档的结构或主题。
从语料库中推断出的潜在结构（Latent Structure）可用于将主题分配给先前不存在于仅用于训练的语料库中的新文档。 
出于这个原因，我们也将此集合称为训练语料库（Training Corpus）。 这个过程不需要人工干预（比如手动给文档打标签） 
--- 因为主题分类是无监督的（Unsupervised）

In [6]:
raw_corpus = list()   # content for the whole data


f = open("input.txt", "r", encoding="utf8")
# f = open("/input/input.txt", "r", encoding="utf8")


# Add content to list (base on “A”, “B”, “C”)
for line in f:
    x = json.loads(line)    
    raw_corpus.append(x["A"])   
    raw_corpus.append(x["B"])
    raw_corpus.append(x["C"])
    

## 利用 Jieba 对语料库中的文档进行分词


In [7]:
# Return a List
for a in range(0, len(raw_corpus)):
    raw_corpus[a] = trans(raw_corpus[a])

## 移除常用词以及分词

In [8]:
# import a stopwrods list
stoplist = [i.strip() for i in open('stopword.txt',encoding='utf-8').readlines()]

In [9]:
# Remove words according to the stoplist
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in raw_corpus]

# Non-stoplist version
# texts = [[word for word in document.lower().split()]
#          for document in raw_corpus]

In [10]:
#计算词频
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
        
    

In [11]:
# 仅保留词频数高于1的词汇
'''
processed_corpus = [[line 1 A], [line 1 B], [line 1 C],
                    [line 2 A], [line 2 B], [line 2 C]
                    ...
                    [line 500 A], [line 500 B], [line 500 C]] 

'''
processed_corpus = [[token for token in text if frequency[token] > 1] for text in texts]





## 构建字典

In [12]:
# 将语料库中的每个词汇与唯一的整数ID相关联
# Create a dictionary
dictionary = corpora.Dictionary(texts)
print(type(dictionary))
print(dictionary)

<class 'gensim.corpora.dictionary.Dictionary'>
Dictionary(4347 unique tokens: ['1', '12015', '2', '2015', '2016']...)


# 2: 向量空间（Vector Space）

为了推断语料库中的潜在结构（Latent Structure），我们需要一种可用于数学操作（比如，加减乘除等运算）的文档表示方法。一种方法是将每个文档表示为向量。有各种用于创建文档的向量表示的方法，其中一个简单的方法是词袋模型(Bag-of-Words Model)。
在词袋模型下，每个文档由包含字典中每个单词的频率计数的向量表示。例如，给定一个包含词汇['咖啡'，'牛奶'，'糖果'，'勺子']的字典，那么，一个由字符串'咖啡 牛奶 糖果 勺子'组成的文档可以用向量表示为[2 ，1，0,0]，其中向量的元素（按顺序）对应文档中出现的“咖啡”，“牛奶”，“糖”和“勺子”。向量的长度是字典中的词汇数。词袋模型的一个主要特性是它完全忽略了编码文档（the Encoded Document ）中的词汇顺序，这就是词袋模型的由来。



In [13]:
# key -> number; value -> words
# See the structure of the dictionary

# print(dictionary.token2id)


### Vector Space Example (将文本用向量表示)

假设我们想要对“通远乡霞山区国家石头专用部门王八蛋周总周总”这个语句进行向量化（请注意，该语句不在我们原来的语料库中）。 我们可以使用dictionary的doc2bow方法为该语句创建词袋表示，该方法返回词汇计数的稀疏表示：

In [14]:
new_doc = "通远乡 霞山区 国家 石头 专用 部门 王八蛋 周总 周总"  #已分词，便于后续处理

# 利用创建好的 dictionary 来匹配每个 word, (id, frequency)  
new_vec = dictionary.doc2bow(new_doc.lower().split()) 

new_vec

# 每个元组中的第一个元素对应字典中的词汇ID，第二个条目对应于该词汇的计数。此向量仅包含实际出现在文档中的词汇。

[(4301, 2), (4308, 1), (4317, 1), (4325, 1), (4327, 1), (4342, 1), (4346, 1)]

## 利用字典将整个原始语料库 (processed_corpus) 转换为向量列表

In [15]:
# txt file 每一篇文章都转换成向量
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]

# bow_corpus
# print(bow_corpus[0])

# 3: 模型 （ TF-IDF , LSI ，LDA ，RP）

## 利用训练好的 Model 将原始文本转好成相应 Model 的向量空间

我们已经对测试语料库进行了向量化，我们可以开始使用models对其进行转换了。 我们使用模型作为抽象术语，指的是从一个文档表示到另一个文档表示的转换。 在gensim中，文档表示为向量，因而模型可以被认为是两个向量空间之间的转换。 从训练语料库中学习这种转换的细节。

一个简单的模型示例是TF-IDF。 TF-IDF模型将向量从词袋表示（Bag-of-Words Representation）转换为向量空间，其中频率计数根据语料库中每个单词的相对稀有度（the relative rarity of each word in the corpus）进行加权。

In [258]:
# scikit-learn
# doc2bow()与在CountVectorizer上调用transform()有类似的作用
# doc2bow()也可以像fit_transform()那样运作


# 训练模型 (TF-IDF, LSI, LDA ,RP)
# 利用训练好的 Model 将原始文本转好成相应 Model 的向量空间

# ------------ TF-IDF ------------
# 初始化 tf-idf 模型，在测试语料库上进行训练
# input: bow_corpus (已经利用词典将每个词语转化成向量列表格式)
tfidf_model = models.TfidfModel(bow_corpus)
print(tfidf_model)      # TfidfModel(num_docs=1500, num_nnz=194228)

# --- 利用模型 (TF-IDF)，将整个语料库中每一个sample转为tfidf表示方法 ---
corpus_tfidf = tfidf_model[bow_corpus]

print("利用 TF-IDF 转换好的样本size == 原样本size", len(corpus_tfidf))
# print(corpus_tfidf[0])


print("\n")


# ------------ LSI ------------

# 初始化 LSI 模型，在测试语料库上进行训练 （# 184, 13）（利用 corpus_tfidf）
lsi_model = models.LsiModel(bow_corpus, id2word=dictionary, num_topics= 12)
print(lsi_model)

# --- 利用模型 (LSI)，将整个语料库中每一个sample转为LSI表示方法 ---
corpus_lsi = lsi_model[corpus_tfidf]
print("利用 LSI 转换好的样本size == 原样本size", len(corpus_lsi))


print("\n")



# --- LDA ---
# 初始化 LDA 模型，在测试语料库上进行训练 （# 13）
lda_model = models.LdaModel(bow_corpus, id2word=dictionary, num_topics= 13)
print(lda_model)

# --- 利用模型 (LDA)，将整个语料库中每一个sample转为LDA表示方法 ---
#### using corpus_tfidf ####
corpus_lda = lda_model[corpus_tfidf]

print("利用 LDA 转换好的样本size == 原样本size", len(corpus_lda))
print(corpus_lda[0])




print("\n")


# ------------ RP (Random Projections) ------------

# 初始化 RP 模型，在测试语料库上进行训练 （利用 corpus_tfidf）
rp_model = models.RpModel(corpus_tfidf, id2word=dictionary, num_topics= 16
                         )
print(rp_model)

# --- 利用模型 (RP)，将整个语料库中每一个sample转为RP表示方法 ---
corpus_rp = rp_model[corpus_tfidf]
print("利用 RP 转换好的样本size == 原样本size", len(corpus_rp))




TfidfModel(num_docs=1500, num_nnz=194228)
利用 TF-IDF 转换好的样本size == 原样本size 1500


LsiModel(num_terms=4347, num_topics=12, decay=1.0, chunksize=20000)
利用 LSI 转换好的样本size == 原样本size 1500


LdaModel(num_terms=4347, num_topics=13, decay=0.5, chunksize=2000)
利用 LDA 转换好的样本size == 原样本size 1500
[(0, 0.012681211), (1, 0.012681254), (2, 0.012681115), (3, 0.012681498), (4, 0.012681212), (5, 0.012681256), (6, 0.012681219), (7, 0.84782505), (8, 0.012681265), (9, 0.0126811825), (10, 0.012681193), (11, 0.012681313), (12, 0.012681206)]


RpModel(num_terms=4347, num_topics=16)
利用 RP 转换好的样本size == 原样本size 1500


In [259]:
# LDA 分类结果 (最具有代表性的5个)
print(lda_model.print_topic(1, topn=5))

0.064*"被告" + 0.040*"1" + 0.037*"原告" + 0.030*"月" + 0.025*"借款"


### 例子: TF-IDF模型将向量从词袋表示 转化成TFIDF向量（的迭代器）

In [260]:
# 对 “通远乡霞山区国家石头专用部门王八蛋周总周总" 进行转换

# ------------ TF-IDF模型返回元组列表 ------------
# print("利用TF-IDF模型返回元组列表，每个元组的第一个元素是词汇ID，第二个条目是TF-IDF加权值.") 

# 每个元组的第一个元素是词汇ID，第二个条目是TF-IDF加权值。 
# tfidf_model[dictionary.doc2bow("通远乡 霞山区 国家 石头 专用 部门 王八蛋 周总 周总".split())]



# ------------ LSI 模型返回元组列表 ------------
# lsi_model[dictionary.doc2bow("通远乡 霞山区 国家 石头 专用 部门 王八蛋 周总 周总".split())]


# ------------ LDA 模型返回元组列表 ------------
# print("利用LDA模型返回元组列表，每个元组的第一个元素是词汇ID，第二个条目是TF-IDF加权值.") 
# lda_model[dictionary.doc2bow("通远乡 霞山区 国家 石头 专用 部门 王八蛋 周总 周总".split())]


# ------------ RP 模型返回元组列表 ------------
# print("利用RP模型返回元组列表，每个元组的第一个元素是词汇ID，第二个条目是TF-IDF加权值.") 
# rp_model[dictionary.doc2bow("通远乡 霞山区 国家 石头 专用 部门 王八蛋 周总 周总".split())]


# 4: 相似度的计算

In [261]:
# ------------ 创建索引 ------------

# tfidf method
index_tfidf = similarities.MatrixSimilarity(corpus_tfidf)
# index_tfidf = similarities.SparseMatrixSimilarity(corpus_tfidf, num_features=len(dictionary.keys()))

# LSI method
index_lsi = similarities.MatrixSimilarity(corpus_lsi)


# LDA method
index_lda = similarities.MatrixSimilarity(corpus_lda)


# RP method
index_rp = similarities.MatrixSimilarity(corpus_rp)


In [262]:
print("index_tfidf", index_tfidf)
print("index_lsi", index_lsi)
print("index_lda", index_lda)
print("index_rp", index_rp)

index_tfidf MatrixSimilarity<1500 docs, 4342 features>
index_lsi MatrixSimilarity<1500 docs, 12 features>
index_lda MatrixSimilarity<1500 docs, 13 features>
index_rp MatrixSimilarity<1500 docs, 16 features>


### Example : 利用 TF_IDF , LSI , LDA 计算文本中 line1 A B C 相似度

In [263]:
# ------------------------ tfidf method ------------------------
print("------ tfidf method ------")

# 第一篇文章的向量
A_tfidf = [corpus_tfidf[0]]

# line-A 和 整个line1 比较
sim_tfidf = [index_tfidf[element][0:3] for element in A_tfidf]
print(sim_tfidf)

print("\n")


# ------------------------ lsi method ------------------------
print("------ lsi method ------")

# 第一篇文章的向量
A_lsi = corpus_lsi[0]

# line-A 和 整个line1 比较
sim_lsi = index_lsi[A_lsi][0:3] 
print(sim_lsi)


# ------------------------ lsi method ------------------------
print("------ lda method ------")

# 第一篇文章的向量
A_lda = corpus_lda[0]
B_lda = corpus_lda[1]
C_lda = corpus_lda[2]

# Cosine similarity
sim_lda_cosine = gensim.matutils.cossim(A_lda, B_lda)

print(sim_lda_cosine)

# Hellinger distance
dense1 = gensim.matutils.sparse2full(A_lda, lda_model.num_topics)
dense2 = gensim.matutils.sparse2full(B_lda, lda_model.num_topics)
sim_hd = np.sqrt(0.5 * ((np.sqrt(dense1) - np.sqrt(dense2))**2).sum())
print(sim_hd)


# ------------------------ rp method ------------------------

# 第一篇文章的向量
A_rp = corpus_rp[0]

# line-A 和 整个line1 比较
sim_rp = index_rp[A_rp][0:3] 
print(sim_rp)






------ tfidf method ------
[array([1.0000001 , 0.00720893, 0.00726172], dtype=float32)]


------ lsi method ------
[ 1.         -0.44931525 -0.16945885]
------ lda method ------
0.030291621010760588
0.8152550790202141
[ 1.         -0.12088701 -0.3288839 ]


## Define Functions for each comparsion example

In [264]:
#################### TF-IDF ####################

# i: acc (start of the list)
# j: acc (end of the list)
# size: size of the list

def tfidf_compare(i, j, size):
    while i < size:  
        
#----------------- Built in sim function ----------------------
        
        # 待比较的文档 （A）
        A = [corpus_tfidf[i]]
    
        # 相似度计算
        j = i + 3
        sims = [index_tfidf[new_vec_tfidf][i:j] for new_vec_tfidf in A]
    
        v1 = sims[0][1]
        v2 = sims[0][2]
                
        if v1 > v2:
            print("B", file=ouf)
        else:
            print("C", file=ouf)
        i = j
    ouf.close()
    
        
        


#################### LSI ####################

# i: acc (start of the list)
# j: acc (end of the list)
# size: size of the list

def lsi_compare(i, j, size):
    while i < size:
        # 待比较的文档 （A）
        A = corpus_lsi[i]
        j = i + 3
        # 相似度计算
        sims = index_lsi[A][i:j] 
        v1 = sims[1]
        v2 = sims[2]
        
        if v1 > v2:
            print("B", file=ouf)
        else:
            print("C", file=ouf)
        i = j
    ouf.close()

    
#################### LDA (Hellinger distance) ####################

# i: acc (start of the list)
# j: acc (end of the list)

def lda_compare(i, j):
    while i < size:
        # 待比较的文档 （A）
        A = corpus_lda[i]
        B = corpus_lda[i + 1]
        C = corpus_lda[i + 2]
        j = i + 3
        # 相似度计算
        
        dense_A = gensim.matutils.sparse2full(A, lda_model.num_topics)
        dense_B = gensim.matutils.sparse2full(B, lda_model.num_topics)
        dense_C = gensim.matutils.sparse2full(C, lda_model.num_topics)

        distance_AB = np.sqrt(0.5 * ((np.sqrt(dense_A) - np.sqrt(dense_B))**2).sum())
        distance_AC = np.sqrt(0.5 * ((np.sqrt(dense_A) - np.sqrt(dense_C))**2).sum())

        
        if distance_AB > distance_AC:
            print("C", file=ouf)
        else:
            print("B", file=ouf)
        i = j
    ouf.close()
    
    
    

    
#################### RP ####################

# i: acc (start of the list)
# j: acc (end of the list)
# size: size of the list

def rp_compare(i, j, size):
    while i < size:
        # 待比较的文档 （A）
        A = corpus_rp[i]
        j = i + 3
        # 相似度计算
        sims = index_rp[A][i:j] 
        v1 = sims[1]
        v2 = sims[2]
        
        if v1 > v2:
            print("B", file=ouf)
        else:
            print("C", file=ouf)
        i = j
    ouf.close()

        

## Start Comparing

In [265]:
ouf = open("output.txt", "w", encoding="utf8")
# ouf = open("/output/output.txt", "w", encoding="utf8")
size = len(raw_corpus)

# ------ TF-IDF ------

# tfidf_compare(0, 0, size)

# ------ LSI ------
# lsi_compare(0, 0, size)


# ------ LDA ------
# lda_compare(0, 0)


# ------ RP ------
rp_compare(0, 0, size)

## Define Evaluation Function

In [266]:
def eval():
    ouf = open("output.txt", "r", encoding="utf8")    
    B = 0
    C = 0
    x = ouf.readlines()
    for i in x:
        if i == 'B\n':
            B = B + 1
        if i == 'C\n':
            C = C + 1
        else: 
            continue

    print("Number of C is ", C)
    print("Number of B is ", B)
    print("Model accuarcy is ", B/(C+B))
ouf.close()   

eval()

Number of C is  248
Number of B is  252
Model accuarcy is  0.504
